In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Assumed values

rf = 0.06
S0 = 12089
K = 12200
T = 50/365
nT = 10
nS = 10
sig = 0.15
delT = T/nT

In [3]:
ass_price = np.zeros((nS,nT))
intrinsic_val = np.zeros((nS,nT))
updated_val = np.zeros((nS,nT))

In [4]:
ass_price[:,0]=S0

for i in range(0,nS):
    for j in range(1,nT):
        ass_price[i,j] = ass_price[i,j-1]*np.exp((rf-sig*2/2)*delT + sig*np.sqrt(delT)*np.random.randn(1))

ass_price


array([[12089.        , 12015.83568361, 11647.0012625 , 11518.03944677,
        11330.0517002 , 11453.50240785, 11483.38532121, 11418.66719037,
        11625.61515499, 11280.38208002],
       [12089.        , 11813.59506918, 11942.60564912, 11878.54268228,
        11726.82068442, 11701.44878764, 11997.06687771, 12039.49886459,
        12047.38497885, 11874.45248056],
       [12089.        , 11982.300242  , 11757.54523485, 11423.59207542,
        11437.65403947, 11296.47065672, 11260.48379962, 11056.88395874,
        10800.19346137, 10900.44943952],
       [12089.        , 12247.81531073, 12154.11632429, 12037.46580408,
        12259.76932693, 12194.00064325, 11867.28305833, 11789.50561697,
        11576.67120753, 11784.84686439],
       [12089.        , 12184.00786032, 12061.83868693, 12316.46796549,
        12100.10136545, 12284.42017361, 12466.3931078 , 12547.56192102,
        12843.43858069, 12526.76156735],
       [12089.        , 12143.90928812, 11961.3311525 , 12163.01062225,
   

In [5]:
for i in range(0,nS):
    for j in range(0,nT):
        intrinsic_val[i,j]=max(ass_price[i,j]-K,0)

intrinsic_val

array([[   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,   47.81531073,    0.        ,    0.        ,
          59.76932693,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,  116.46796549,
           0.        ,   84.42017361,  266.3931078 ,  347.56192102,
         643.43858069,  326.76156735],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0

In [6]:
updated_val[:,nT-1]=intrinsic_val[:,nT-1]
updated_val

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        , 326.76156735],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  

In [40]:
from sklearn.linear_model import LinearRegression

In [51]:
for i in range(nT-2,0,-1):
    indicator = intrinsic_val[:,i]>0
    index = np.where(indicator)[0]
    
    pv_wait = np.zeros(nS)
    for j in range(0,nS):
        for k in range(i+1,nT):
            pv_wait[j]=pv_wait[j]+updated_val[j,k]*np.exp(-rf*(k-i)*delT)
    
    p = ass_price[index,i]
    pp = ass_price[index,i]**2  
    
    regressor = LinearRegression()
    X_train = np.concatenate([np.ones(len(index)).reshape(-1,1),p.reshape(-1,1),pp.reshape(-1,1)],axis=1)
    y = pv_wait[index]
    regressor.fit(X_train, y)
    
    y_pred = np.zeros(nS)
    y_pred[index]=regressor.predict(X_train)
    
    for j in range(0,nS):
        if(intrinsic_val[j,i]>y_pred[j]):
            updated_val[j,i]=intrinsic_val[j,i]
            for k in range(i+1,nT):
                updated_val[j,k]=0
    

In [52]:
updated_val

array([[   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
          59.76932693,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
         643.43858069,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0

In [55]:
pv_wait = np.zeros(nS)
for j in range(0,nS):
    for k in range(0,nT):
        pv_wait[j]=pv_wait[j]+updated_val[j,k]*np.exp(-rf*(k)*delT)
            
call = np.mean(pv_wait)
call

238.79816212397304

In [56]:
np.mean([0,1,2])

1.0